# MultiTask and Instrumentation Data I/O

J. M. Hughes. Real World Instrumentation with Python, O'Reilly Media, December 2010, ISBN 978-0-596-80956-0

* CHAPTER 11 Instrumentation Data I/O

  * Data I/O: Acquiring and Writing Data



## 1 Blocking Versus Nonblocking Calls 

Now it’s time to introduce some concepts that you will need to use later to build robust and reliable software. We’ll start with a discussion of blocking and nonblocking function calls, and then take a look at some basic techniques for handling errors.

One way to describe the behavior of a function or method is `in terms of how quickly it will return after it has been invoked`. Some only return `after` a result of some type is obtained, while others may return `immediately` without waiting for something else downstream to produce a particular response.

In other words, functions is

* **blocking** : the calling code must **wait** for a response 

* **nonblocking** : the call returns **immediately**, usually with a response that indicates success or failure

Actually, all software functions (and methods, too) can be classified as either `blocking or nonblocking`, and the majority of functions within a typical software application are of the blocking variety—that is, they don’t return until the intended action is complete or an error is detected. 

### 1.1 Blocking 

You can see this in the **message sequence chart (MSC)(消息序列图)** shown in `Figure 11-9`.

Here we have `Function1()` calling `Function2()`, which in turn calls `Function3()` and finally `Function4()`. The time required for `Function1()` to receive a response from `Function2()` is dependent on how long it takes for functions 2, 3, and 4 to complete their processing and return. During this entire time, **Function1()** is blocked.(In an MSC diagram, events in a function or process occur in a top-to-bottom order,and transactions between functions or processes are the horizontal lines.)

![data-io-msc](./img/data-io-msc.jpg)

>**Message sequence charts**
The message sequence chart (MSC) is defined by the guidelines document Z.120, maintained by the `International Telecommunication Union (ITU)`. In its current form, an MSC is a powerful tool for modeling command-response transactions between multiple entities. The UML equivalent of the MSC is the **sequence diagram**

**Blocking** allows functions to maintain **synchronization(同步) and honor the intended flow of execution** through the code. The action or data that the call is requesting may or may **not be available** at the time the call is made, so a `blocking` call will `wait` for the other end to respond in some fashion before returning to the caller. As a side effect, it will also effectively `suspend your application` until it returns.

**The type of blocking** we’re most interested in is when an application process is forced to **wait for an interface**, which in turn waits for a `hardware device to respond`. 

#### 1.1.1 Blocking IO

In [1]:
import time
import psutil

def get_data():
    time.sleep(2)
    return psutil.cpu_percent()

def cpu_monitor_sync():
    """ blocking I/O call"""
    return get_data()
  
for i in  range(3):
   # blocking call
    time1=time.localtime(time.time())
    print("Begin of the call at the second:",str(time1.tm_sec))
    values=cpu_monitor_sync()
    print("cpu_percent:",values)
    time2=time.localtime(time.time())
    print("End of the call at  the second:",str(time2.tm_sec))  


Begin of the call at the second: 34
cpu_percent: 13.4
End of the call at  the second: 36
Begin of the call at the second: 36
cpu_percent: 11.1
End of the call at  the second: 38
Begin of the call at the second: 38
cpu_percent: 5.3
End of the call at  the second: 40


#### 1.1.2 The Simplest Monitor using Matplotlib.Plot

```python
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
```

Set `interval=1000`
```python
ani = FuncAnimation(fig, update,init_func=init, blit=True,interval=1000)
```
Test

1. The times of Blocking IO > the `interval=1000`

```python
time.sleep(2)
```

2. The times of Blocking IO < the `interval=1000`

```python
time.sleep(0.1)
```    

In [13]:
%%file ./code/concurrency/demo_blocking_io_cpu_simplest_monitor_matplotlib.py
import time
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
import psutil
from matplotlib.animation import FuncAnimation

def virtual_interface_data(tag):
    # time.sleep(0.1)
    try:
        if tag=="CPU_PERCENT":
            value=psutil.cpu_percent()
        elif tag=="MEM_PERCENTT":
            value=psutil.virtual_memory().percent
        rc=1    
    except:
        rc,value=0,None  
    return (rc,value)        
            
tag="CPU_PERCENT"
y = deque()

columns = ()
col_labels = ['Tag', 'Unit', 'Value']
table_vals = [[tag,"%",""]]


fig, ax = plt.subplots()
ax.set_title("The Simplest Monitor:"+tag)
ln, = plt.plot([], [], 'b-o')
str_cursecond=str(time.localtime(time.time()).tm_sec)   
time_text = ax.text(0.5, 80, "")

tbl = ax.table(cellText=table_vals,
               colLabels=col_labels,
               colWidths=[0.2] * 3,
               cellLoc='center',
               loc='best')

def init():
    ax.set_xlim(0, 9)
    ax.set_ylim(0, 100)
    return ln,

def update(frames):
    rc,value = virtual_interface_data(tag)
    if len(y) < 10:
        y.append(value)
    else:
        y.popleft()
        y.append(value)

    str_curtime=time.strftime("%F %H:%M:%S", time.localtime(time.time()))
    time_text.set_text("Time:"+str_curtime)
    
    table_vals = [[tag,"%",str(value)]]
    tbl = ax.table(cellText=table_vals,
               colLabels=col_labels,
               colWidths=[0.2] *3,
               cellLoc='center',
               loc='best')

    ln.set_xdata(np.arange(len(y)))
    ln.set_ydata(np.array(y))
    return ln,time_text, tbl

ani = FuncAnimation(fig, update,init_func=init, blit=True,interval=1000)
plt.show()

Overwriting ./code/concurrency/demo_blocking_io_cpu_simplest_monitor_matplotlib.py


In some cases it `may not matter` if a blocking call `waits for a bit` before returning to the caller, and allowing this is more convenient than writing the necessary code to support continual query and retry actions.

But, there is a warning in order here: when working with I/O devices, **a blocking call without a timeout of some sort** can potentially **hang forever**. This is usually a bad thing, and often the only way to get out of the situation is to shut down Python and restart the application. If your code is running on an unattended machine somewhere in the middle of nowhere, a fault that hangs a blocking call can be really, really bad.


### 1.2 Threads

The programs often need to perform several tasks simultaneously. For example, a program may:

* Execute procedures that accomplish intermediate tasks in **parallel** and so improve performance

* Process user **input** while carrying on time-consuming data communication or real-time operations `“in the background”`

Different tasks are performed simultaneously by the **concurrent execution** of parts of the program. Especially on modern multiprocessor systems — including multicore processors,of course — it is increasingly important for programs to take advantage of concurrency to use the system’s resources efficiently.

When you start a program, the operating system creates a new **process(进程)** in which the program is executed. A process consists of one or more **threads(线程)**. Each thread is a partial process that executes a sequence of instructions **independently** of other parts of the process.

When the process begins, its **main thread** is active. From then on, any running thread can launch other threads.

All threads that have been started but not yet ended are `terminated when the process terminates`.

Every process has its own address space in memory, and has other exclusive resources, such as open files. All the threads of a process inherit its resources. Most significantly,several **threads** in one process `share` the **same address space.**

Because the threads of a given process use the **same address space**, they share their global and static data. That means, however, that two different threads can access the same memory locations concurrently. This situation is called a **race condition(竞争条件)**. To prevent **inconsistencies** in shared data, the programmer must **explicitly `synchronize` different threads**’ writing operations or reading and writing operations if they use the same locations in memory.


#### 1.2.1 threading Module

The threading module provides APIs for managing several threads of execution, which allows a program to run multiple operations concurrently in the same process space.

The simplest way to use a **Thread** is to instantiate it with a **target** function and call **start()** to let it begin working

In [10]:
import threading
import time
import datetime

def worker(num):
    """thread worker function"""
    time.sleep(2.0)
    str_curtime=datetime.datetime.now().strftime('%F %H:%M:%S.%f')
    print('\nWorker {} at {}'.format(num,str_curtime),end="")

for i in range(5):
    t = threading.Thread(target=worker,args=(i,))
    t.start()


Worker 0 at 2019-05-01 01:52:04.899010
Worker 2 at 2019-05-01 01:52:04.906013
Worker 3 at 2019-05-01 01:52:04.915010
Worker 4 at 2019-05-01 01:52:04.922025
Worker 1 at 2019-05-01 01:52:04.931988

#### 1.2.2 Daemon thread

Up to this point, the example programs have **implicitly** waited to exit until all threads have completed their work.

Sometimes, however, programs spawn a thread as a **daemon(守护/后台）** that runs without blocking the main program from exiting. 

**Daemon threads** are useful for services where there may not be an easy way to interrupt the thread, or where letting the thread
die in the middle of its work does not lead to loss or corruption of data(主线程退出时,不需要等待daemon子线程完成）

To mark a thread as a daemon, pass `daemon=True` when constructing it or call its `setDaemon()` method with `True`. The default is for threads to not be daemons.

>The **logging** module supports `embedding the thread name` in every log message using the formatter code %(threadName)s.Including thread names in log messages makes it possible to trace those messages back to their source.

In [43]:
%%file ./code/concurrency/threading_daemon.py
import threading
import time
import logging

def daemon():
    logging.debug('Starting')
    time.sleep(2)
    logging.debug('Exiting')

def non_daemon():
    logging.debug('Starting')
    logging.debug('Exiting')

logging.basicConfig(
    level=logging.DEBUG,
    format='(%(threadName)-10s) %(message)s',
)

d = threading.Thread(name='daemon', target=daemon, daemon=True)
t = threading.Thread(name='non-daemon', target=non_daemon)
d.start()
t.start()

Overwriting ./code/concurrency/threading_daemon.py


In [44]:
!python ./code/concurrency/threading_daemon.py

(daemon    ) Starting
(non-daemon) Starting
(non-daemon) Exiting


The output from this code does not include the **"Exiting"** message from the **daemon** thread

since all of the non-daemon threads (including the main thread) exit before the daemon thread wakes up from the `time.sleep()` call.

#### 1.2.3 join() Method

```python
join(timeout=None)
```
Wait until the thread terminates. This blocks the calling thread until the thread whose `join()` method is called terminates

* either normally or through an unhandled exception 

* until the optional timeout occurs.

Use the **join()** method to wait until a `daemon` thread has completed its work, 

In [51]:
%%file ./code/concurrency/threading_daemon_join.py
import threading
import time
import logging

def daemon():
    logging.debug('Starting')
    time.sleep(3)
    logging.debug('Exiting')

def non_daemon():
    logging.debug('Starting')
    logging.debug('Exiting')

logging.basicConfig(
    level=logging.DEBUG,
    format='(%(threadName)-10s) %(message)s',
)

start_time = time.time()
d = threading.Thread(name='daemon', target=daemon, daemon=True)
t = threading.Thread(name='non-daemon', target=non_daemon)
d.start()
t.start()
d.join()
t.join()
print(threading.current_thread().name+" Exiting!")
print('Total Time：', time.time()-start_time)

Overwriting ./code/concurrency/threading_daemon_join.py


In [52]:
!python ./code/concurrency/threading_daemon_join.py

MainThread Exiting!
Total Time： 3.0022974014282227


(daemon    ) Starting
(non-daemon) Starting
(non-daemon) Exiting
(daemon    ) Exiting


#### 1.2.4 Subclassing Thread

At start-up, a Thread does some basic initialization and then calls its **run()** method, which in turn calls the `target` function passed to the constructor.

To create a subclass of Thread, override `run()` to do whatever is necessary.

In [11]:
%%file ./code/concurrency/threading_subclass.py
import threading
import logging
import time
import datetime

class MyThread(threading.Thread):
    
    def run(self):
        time.sleep(2.0)
        str_curtime=datetime.datetime.now().strftime('%F %H:%M:%S.%f')
        logging.debug('running at '+ str_curtime)
        
logging.basicConfig(
    level=logging.DEBUG,
    format='(%(threadName)-10s) %(message)s',)

def main():
    for i in range(5):
        t = MyThread()
        t.start() 

if __name__ == "__main__":
    main()

Overwriting ./code/concurrency/threading_subclass.py


In [12]:
!python ./code/concurrency/threading_subclass.py

(Thread-1  ) running at 2019-05-01 01:52:50.656306
(Thread-2  ) running at 2019-05-01 01:52:50.656306
(Thread-5  ) running at 2019-05-01 01:52:50.657240
(Thread-3  ) running at 2019-05-01 01:52:50.657240
(Thread-4  ) running at 2019-05-01 01:52:50.657240


The thread’s **start** method `automatically` invokes its **run** method. 

The **Thread** class maintains an instance variable for the thread’s name and includes the associated methods `getName` and `setName`. 

The Table lists **some important Thread methods.**

|Thread Method| What It Does|
|:---------:|:--------:|
|__init__(name = None)| Initializes the thread’s name.|
|getName()| Returns the thread’s name.|
|setName(newName)| Sets the thread’s name to newName.|
|run() |Executes when the thread acquires the CPU.|
|start()| Makes the new thread ready. Raises an exception if run more than once. |
| isAlive()| Returns True if the thread is alive or False otherwise |


### 1.3 Non-blocking

One way to deal with this is to use **nonblocking** function calls. This entails some `additional` code, but it’s very useful when dealing with network communications and data acquisition. 

**Threading** is a technique for decoupling tasks which are not sequentially dependent. Threads can be used to improve the responsiveness of applications that accept user input while other tasks run in the background. 

* A related use case is running I/O in parallel with computations in another thread.

This is shown in Figure 11-10. Notice that there is a **timer** symbol in this diagram. This means that if the hardware does not respond within some preset period of time, `the interface process will terminate and return an error`

![data-io-io](./img/data-io-io.jpg)

We’ll look at some ways to realize **nonblocking** function using **Concurrency Programming**.

To implement concurrency, it is necessary to think and code differently; in the following sections, we'll demonstrate techniques and best practices to implement **robust concurrent applications**.

#### 1.3.1 Non-blocking with timeout using threading

In [ ]:
%%file ./code/concurrency/nonblocking_io_cpu.py
"""
  https://blog.csdn.net/MeteorCountry/article/details/82765919
"""
import psutil
import time
import threading

class IOThread(threading.Thread): 
    
    def __init__(self,target,args=()):  
        super(IOThread,self).__init__()
        self.func = target
        self.args = args
        
    def run(self):
        self.result = self.func(*self.args) 
            
    def get_result(self):
        try:   
            return self.result
        except Exception:
            return None

def timeout_decor(timeout): 
    """ timeout: max time,s  
        return: if timeout ,return None 
    """ 
    def functions(func): 
    
        def run(*params):
            thre_func = IOThread(target=func,args=params)
            # if timeout,main thread stop,then the iothread is stoped 
            thre_func.setDaemon(True)
            thre_func.start()
            sleep_num = int(timeout // 1)
            sleep_nums = round(timeout % 1, 1) 
            for i in range(sleep_num):
                time.sleep(1)
                infor = thre_func.get_result()
                if infor:
                    return infor
            time.sleep(sleep_nums)
            return thre_func.get_result()       
            
        return run
    
    
    return functions

def get_data_with_timeout(timeout):
    @timeout_decor(timeout)
    def get_data():
        time.sleep(2)
        return psutil.cpu_percent()        
    
    value=get_data()
    if value is not None:
        rc=1 # ok
    else: 
        rc=0 # timeout
    return (rc,value)    

if __name__ == '__main__':
    timeout=3.0                   
    rc,value=get_data_with_timeout(timeout)
    print(rc,value)

In [ ]:
%%file ./code/concurrency/demo_nonblocking_io_cpu.py
import sys  
sys.path.append('./code/concurrency/')  

from nonblocking_io_cpu import get_data_with_timeout

if __name__ == '__main__':
    timeout=3.0                   
    rc,value=get_data_with_timeout(timeout)
    print(rc,value)

#### 1.3.2 Demo NonBlocking IO using Matplotlib.Plot


Set `interval=1000`
```python
ani = FuncAnimation(fig, update,init_func=init, blit=True,interval=1000)
```
**Set IO time.sleep > interval=1000**

```python
time.sleep(2)
```
Test

1. The IO is timeout `

```python
 rc,value = get_data_with_timeout(1)
```

2. The IO is fast`

```python
time.sleep(0.1)
```

**Set IO time.sleep < interval=1000**

```python
time.sleep(0.5)
```
Test

1. The IO is timeout `

```python
 rc,value = get_data_with_timeout(1)
```

2. The IO is fast`

```python
time.sleep(0.1)
```    
```    

In [14]:
%%file ./code/concurrency/demo_nonblocking_io_cpu_matplotlib.py
import time
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
import psutil
from matplotlib.animation import FuncAnimation

import sys  
sys.path.append('./code/concurrency/')  
from nonblocking_io_cpu import get_data_with_timeout

fig, ax = plt.subplots()
ln, = plt.plot([], [], 'b-o')
str_cursecond=str(time.localtime(time.time()).tm_sec)   
time_text = ax.text(0.5, 80, "")

y = deque()

def init():
    ax.set_xlim(0, 10)
    ax.set_ylim(1, 100)
    return ln,

def update(frames):
    rc,value = get_data_with_timeout(3)
    if len(y) < 10:
        y.append(value)
    else:
        y.popleft()
        y.append(value)
    str_curtime=time.strftime("%F %H:%M:%S", time.localtime(time.time()))  
    if value is None:
        str_cursecond=str_cursecond+" (Timeout)"
    time_text.set_text("Time:"+str_curtime)

    ln.set_xdata(np.arange(len(y)))
    ln.set_ydata(np.array(y))
    return ln,time_text

ani = FuncAnimation(fig, update,init_func=init, blit=True,interval=1000)
plt.show()


Overwriting ./code/concurrency/demo_nonblocking_io_cpu_matplotlib.py


## 2 Data I/O Methods

Now that we’ve seen what blocking and nonblocking functions entail, let’s look at how these concepts are involved with various operational modes of interface I/O. We’ll start with the simplest form, **on-demand I/O**（请求式I/O）, then proceed to **polled I/O**(轮询I/O）, and finally take a quick a look at **multithreaded I/O.**（多线程I/O）

### 2.1 On-demand data I/O 

As I stated earlier, the two most obvious ways to move data into or out of your application are just a matter of reading from or writing to a port or device. 

When sending (writing) data using a serial (RS-232 or RS-485) or GPIB-type interface, there usually is `no need to worry about the use of a blocking call`. In the case of an RS-232 interface that does not use hardware handshaking, the data is sent out through the hardware port `immediately`. 

An RS-485 interface with `a single master and multiple listeners` should never block on a write by the master device, but the listeners may be unresponsive for a period of time. GPIB can also get into a situation where there are no listeners responding to the sender, but most GPIB interface APIs and the associated hardware can detect this and return an error code. 

Writing to a hardware interface API for a device such as a PCI interface card is usually not a problem in terms of blocking, but the call might still return an error code if something is amiss.

If your software uses `on-demand calls` to read data, they should be `blocking calls`, and your software should always check the return codes. If `timeout parameters` are available for a blocking function call they should definitely be used, but not every API provides blocking calls with timeouts (perhaps it was assumed that a timeout couldn’t possibly happen). 

For those situations you’ll need to use a **nonblocking** version of the API function and employ a different approach to **implement a `timeout` in your own software**.

In [ ]:
import sys  
sys.path.append('./code/concurrency/')  

from nonblocking_io_cpu import get_data_with_timeout

rc,value=get_data_with_timeout(4)
print(rc,value)

### 2.2 Polled data I/O 

**A `nonblocking` call will return `immediately`**, and its return code or return value will (hopefully) let the caller know whether or not it succeeded. A nonblocking call can be used to `avoid an I/O hang`, but it requires more code to support it. 

For example, let’s assume that the API we’re using has both blocking and nonblocking versions of I/O functions to read data from a device, or perhaps that the I/O functions have a parameter that can be set to control blocking. You can then put a nonblocking call into a loop that also checks for a timeout, like this:

In [ ]:
import time
from nonblocking_io_cpu import get_data_with_timeout

def GetData(tmax=3):
    checking = True
    tstart = time.time()
    while checking:
        rc,value = get_data_with_timeout(tmax)
        if rc == 1:
            break
        else:
            time.sleep(0.05) # wait 50 ms between checks
    return rc, value

rc,value=GetData(tmax=3)
print(rc,value)

This is an example of **polling**: this function will attempt to get data from a specific data acquisition device by continually polling the port (using the **`get_data_with_limit()`** function call) until valid data appears. 

In between each read attempt it will `sleep` for 50 milliseconds. The delay is mainly for the benefit of the device being read, as many devices can’t tolerate being `hammered continuously for data`. 

In order to actually have a polling function that doesn’t cause the rest of an application to **suspend** while it’s active, you need to use **a thread**.

### 2.3 Acquiring data using a thread with Callbacks

So far we’ve looked at `on-demand` and `polled` data I/O. Now let’s take a quick look at how we might check for incoming data `without bogging down the entire system` in a continuous polling loop. 

**Callbacks**

One of the simplest ways to accomplish this behavior is through **callbacks**. The strategy is quite similar to what we do when we request a cab.

Imagine that you are at a restaurant and you've had a few drinks. It's raining outside, and you'd rather not take the bus; therefore, you request a taxi and ask them to `call when they're outside` so that you can come out, and you don't have to wait in the rain.

What you did in this case is request a taxi (that is, the slow resource) but `instead of waiting outside until the taxi arrives`, you provide your number and instructions (**callback**) so that you can come outside `when they're ready` and go home.

This technique of **`registering` callbacks for execution in response to certain events** is commonly called the ***Hollywood principle(好莱坞原则)**. This is because, after an audition for a role at Hollywood, you may be told  <b style="color:blue">"Don't call us, we'll call you"</b>, meaning that they won't tell you if they chose you for the role immediately, but `they'll call you in case they do`(以通知替代轮询).


There is one API function `GetData()`. It is assumed that these exist as part of the API for the data acquisition hardware, and they do what their names imply. Also, the type of data being acquired isn’t specified, primarily because it doesn’t really matter for this example. It could be anything, just so long as the specified number of samples are acquired and no errors occur:

In [ ]:
import time
import threading

class AcqData:
    
    def __init__(self, timeout):
        self.timeout = timeout
        self.dvals = [] # list for acquired data values
        self.dsamps = 0 # number of values actually read
        self.get_rc = 0 # 0 is OK, negative value is an error
        self.get_done = False # True if thread is finished
   
    def _get_data(self, numsamples):
        cnt = 0
        acqfail = False
        
        while not acqfail:
            self.get_rc, dataval = GetData(self.timeout)
            if self.get_rc == 1:
                self.dsamps = cnt + 1
                self.dvals.append(dataval)
                cnt += 1
                if cnt >= numsamples:
                    break
            else:
                acqfail = True
        
        self.get_done = True

    def StartDataSamples(self, samplecnt):
        try:
            acq_thread = threading.Thread(target=self._get_data,args=(samplecnt,))
            acq_thread.start()
            #acq_thread.join()
        except Exception as e:
            print("Acquire fault: {}".format(str(e)))

    def GetDataSamples(self):
        if self.get_done == True:
            return (self.get_rc, self.dsamps, self.dvals)
        else:
            return (None, 0, 0)

This bit of code uses `a thread`, in the form of the function `_get_data()`, to continuously read the external device to obtain some number of data samples. Notice that `the hypothetical API function GetData()` supports the use of a `timeout` parameter, and we can assume that it will return an error code if a timeout does occur.

The key things in this simple example are 

* **how the thread is created**, and 

* **how we can check to see if the data acquisition is complete**. 

Python’s threading library includes a thread object method called **join()**, which accepts an optional timeout parameter and
is typically used to block the execution of one thread while it is waiting for another to complete. 



In this case **we won’t use `join()`**, so the thread is allowed to run on its own. 

The accessor function `GetDataSamples()` checks the variable `self.get_done` to determine if the thread has finished. If so, `GetDataSamples()` will return the data collected.If the thread is still running, it will return a 3-tuple with the first item set to `None`.It is up to the caller to determine if the sample count returned matches the sample count requested.

In [ ]:
AcqDataClient= AcqData(3)
samplecnt=3
AcqDataClient.StartDataSamples(samplecnt)
while True:
    get_rc,dsmaps,dvals=AcqDataClient.GetDataSamples()
    print("AcqData ON -- ")
    time.sleep(1)
    if get_rc is not None:
        print(get_rc,dsmaps,dvals)
        break


This is just one way to do this, but it illustrates a fundamental issue that is often encountered when working with threads; namely, 

* **at what point does the program come to a halt and wait for something else to finish what it’s doing?**

In a program that is designed to run continuously, this can be dealt with by placing the call to `GetDataSamples()` in a single main loop in the application. This allows it to be checked each time through the loop if data is expected, with the results read back if they are available. Otherwise, the program could just continue to use the last known results.


### 2.4 Communicating Between Threads

**Problem**

You have multiple threads in your program and you want to safely **communicate or exchange data** between them.

One such interaction is the **producer/consumer** relationship. 

**Solution**

Perhaps the safest way to send data from one thread to another is to use a **Queue** from the **queue** library. To do this, you create a Queue instance that is shared by the threads.

Threads then use `put()` or `get(`) operations to add or remove items from the `queue`. 

>The principal challenge of multi-threaded applications is coordinating threads that share data or other resources. To that end, the threading module provides a number of synchronization primitives including locks, events, condition variables, and semaphores.
While those tools are powerful, minor design errors can result in problems that are difficult to reproduce.
>
>So, the preferred approach to task coordination is to concentrate all access to a resource in a single thread
and then use the queue module to feed that thread with requests from other threads. Applications using
Queue objects for inter-thread communication and coordination are easier to design, more readable, and
more reliable.

#### 2.4.1 Queue
**Queue** instances already have all of the required `locking`, so they can be safely shared by as many threads as you wish.

>[queue — A synchronized queue class](https://docs.python.org/3/library/queue.html)
>The queue module implements multi-producer, multi-consumer queues. It is especially useful in threaded programming when information must be exchanged safely between multiple threads

When using `queues`, it can be somewhat tricky to coordinate the shutdown of the producer and consumer. A common solution to this problem is to rely on a special sentinel value, which when placed in the queue, causes consumers to terminate.

If a thread needs to know **immediately** when a consumer thread has processed a particular item of data, you should pair the sent data with an **Event** object that allows the producer to monitor its progress. 




In [ ]:
from queue import Queue
from threading import Thread, Event
import time
import sys  
sys.path.append('./code/')  

from nonblocking_io_cpu import get_data_with_timeout

def DataProducer(out_q):
    tmax=3
    checking = True
    tstart = time.time()
    while checking:
        (rc,value) = get_data_with_timeout(tmax)
        if rc == 1:
            # Make an (data, event) pair and hand it to the consumer
            evt = Event()
            out_q.put(((rc,value), evt))
            # Wait for the consumer to process the item
            evt.wait()
            break
        else:
            time.sleep(0.05) # wait 50 ms between checks

# A thread that consumes data
def DataConsumer(in_q):
    while True:
        # Get some data
        rcvalue, evt = in_q.get()
        print(rcvalue[0],rcvalue[1])
        # Indicate completion
        evt.set()

# Create the shared queue and launch both threads
q = Queue()
t1 = Thread(target=DataConsumer, args=(q,))
t2 = Thread(target=DataProducer, args=(q,))
t1.start()
t2.start()
# Wait for all produced items to be consumed
q.join()

#### 2.4.2 Timer

**Timer Objects**

This class represents an action that should be run only after a certain amount of time has passed — a timer. `Timer` is a subclass of `Thread` and as such also functions as an example of creating custom threads.

Timers are started, as with threads, by calling their `start(`) method. The timer can be stopped (before its action has begun) by calling the `cancel()` method. The interval the timer will wait before executing its action may not be exactly the same as the interval specified by the user.

In [ ]:
from queue import Queue
from threading import Thread,Timer
import time

import sys  
sys.path.append('./code/concurrency/') 

from nonblocking_io_cpu import get_data_with_timeout

def PeriodDataProducer(delay,out_q):
    tmax=3
    rc,value= get_data_with_timeout(tmax)
    if rc == 1:
        out_q.put((rc,value))
    else:
        pass
       
    t=Timer(delay, PeriodDataProducer,(delay,out_q))
    t.start()

def DataConsumer(in_q):
    while True:
        rcvalue = in_q.get()
        print(rcvalue[0],rcvalue[1])
        
q = Queue()
p=PeriodDataProducer(2,q)
c= Thread(target= DataConsumer, args=(q,)) 
c.start()
q.join()

### 2.5  Using  Future and  Pebble 

We will program **nonblocking** IO interface function using **Future** and **Pebble** package

Pebble provides a neat API to manage threads and processes within an application.
```
>python -m pip install Pebble
```
https://github.com/noxdafox/pebble

#### 2.5.1 Non-blocking  IO

In [6]:
%%file ./code/concurrency/nonblocking_io_cpu_pepple.py
import time
import psutil
import pebble 
from concurrent.futures import TimeoutError

def get_data():
    time.sleep(0.5)
    return psutil.cpu_percent()

def get_data_with_timeout(timeout=1):
    pool = pebble.ProcessPool(max_workers=1)
    future = pool.schedule(get_data, args=(), timeout=timeout)
    try:
        result = future.result()
        rc = 1
        value = result
    except TimeoutError:
        rc = 0
        value = None
    except Exception as error:
        rc = 0
        value = None
        print(error)
    return (rc, value)


if __name__ == '__main__':
    rc,value = get_data_with_timeout(1)
    print(rc,value)


Overwriting ./code/concurrency/nonblocking_io_cpu_pepple.py


In [7]:
%%file ./code/concurrency/demo_nonblocking_io_cpu_pepple.py
import sys  
sys.path.append('./code/')  

from nonblocking_io_cpu_pepple import get_data_with_timeout

if __name__ == '__main__':
    timeout=3
    rc,value=get_data_with_timeout(timeout)
    print(rc,value)

Overwriting ./code/concurrency/demo_nonblocking_io_cpu_pepple.py


#### 2.5.2 Demo Non-blocking IO with Matplotlib.plot

Set `Delay=1`
````python
 rc, value = get_data_with_timeout(delay)
```
Test

1. time.sleep < timeout

```python
def get_data():
    time.sleep(0.5)
    return psutil.cpu_percent()
```
2. time.sleep > timeout

```python
def get_data():
    time.sleep(2)
    return psutil.cpu_percent()
```


In [15]:
%%file ./code/concurrency/demo_nonblocking_io_cpu_pepple_loops_matplotlib.py
from queue import Queue
from threading import Thread, Timer
import time
from collections import deque
import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.append('./code/concurrency/')

from nonblocking_io_cpu_pepple import get_data_with_timeout


def PeriodDataProducer(delay,out_q):
    rc, value = get_data_with_timeout(delay)
    if rc == 1:
        out_q.put((rc, value))
    else:
        out_q.put((rc, value))

    t = Timer(delay, PeriodDataProducer, (delay, out_q))
    t.start()


def DataConsumerPlot(in_q,npoints):
    y = deque()
    plt.figure()
    plt.title("The Simplest CPU Percent Monitor")
    lines, = plt.plot([], [], "b-o")
    time_text = plt.text(0.5, 80, "")
    plt.xlim(0, npoints-1)
    plt.ylim(0, 100)

    def DataConsumer(in_q):
        while True:
            rcvalue = in_q.get()
            if len(y) < 10:
                y.append(rcvalue[1])
            else:
                y.popleft()
                y.append(rcvalue[1])

            lines.set_xdata(np.arange(len(y)))
            lines.set_ydata(np.array(y))
            str_curtime=time.strftime("%F %H:%M:%S", time.localtime(time.time()))   
            if rcvalue[1] is None:
                str_cursecond=str_cursecond+" (Timeout)"
            time_text.set_text("Time:"+str_curtime)
            plt.draw()

    c = Thread(target=DataConsumer, args=(in_q,))
    c.start()
    plt.show()


if __name__ == '__main__':
    q = Queue()
    delay = 1
    p = PeriodDataProducer(delay, q)
    npoints = 10
    DataConsumerPlot(q,npoints)
    q.join()


Overwriting ./code/concurrency/demo_nonblocking_io_cpu_pepple_loops_matplotlib.py


### 2.6 Reactive programming

**Reactive programming(响应式编程)** is a paradigm that aims at building better concurrent systems. Reactive applications are designed to comply with the requirements exemplified by the reactive manifesto:

* **Responsive**: The system responds **immediately** to the user.

* **Elastic**: The system is capable of handling different levels of load and is able to adapt to accommodate increasing demands.

* **Resilient**: The system deals with failure gracefully. This is achieved by modularity and avoiding having a single point of failure.

* **Message driven**: The system should not block and take advantage of `events` and `messages`. A message-driven application helps achieve all the previous requirements.

As you can see, the intent of reactive systems is quite noble, but how exactly does reactiveprogramming work? In this section, we will learn about the principles of reactive programming using the **RxPy** library.

```
python -m pip install rx
```

>The RxPy library is part of [ReactiveX](http://reactivex.io), which is a project that implements reactive programming tools for a large variety of languages.

```python
#from nonblocking_io_cpu import get_data_with_timeout
from nonblocking_io_cpu_pepple import get_data_with_timeout
```

Set `Delay=1`
````python
 rc, value = get_data_with_timeout(delay)
```
Test

1. time.sleep < timeout

```python
def get_data():
    time.sleep(0.5)
    return psutil.cpu_percent()
```
2. time.sleep > timeout

```python
def get_data():
    time.sleep(2)
    return psutil.cpu_percent()
```


In [16]:
%%file  ./code/concurrency/demo_nonblcoking_io_cpu_reactive_matplotlib.py
from rx import Observable
import numpy as np
import matplotlib.pyplot as plt
import time

import sys  
sys.path.append('./code/concurrency/')

#from nonblocking_io_cpu import get_data_with_timeout
from nonblocking_io_cpu_pepple import get_data_with_timeout

intervalTime=100
delay=1
cpu_data = (Observable
            .interval(intervalTime) 
            .map(lambda rc,value: get_data_with_timeout(delay))
            .publish())

cpu_data.connect()

def monitor_cpu(npoints):
    plt.figure()
    plt.title("The Simplest CPU Percent Monitor")
    lines, = plt.plot([], [],"b-o")
    time_text = plt.text(0.5, 80, "")
    plt.xlim(0, npoints-1)
    plt.ylim(0, 100)
    cpu_data_window = cpu_data.buffer_with_count(npoints, 1)
    
    def update_plot(cpu_readings):
        lines.set_xdata(np.arange(len(cpu_readings)))
        lines.set_ydata(np.array(cpu_readings)[:,1])
        str_curtime=time.strftime("%F %H:%M:%S", time.localtime(time.time()))  
        
        if np.array(cpu_readings)[-1,1] is None: 
            str_cursecond=str_cursecond+" (Timeout)"
        
        time_text.set_text("Time:"+str_curtime)
        plt.draw()
    
    cpu_data_window.subscribe(update_plot)
    plt.show()

if __name__ == '__main__':
    monitor_cpu(10)


Overwriting ./code/concurrency/demo_nonblcoking_io_cpu_reactive_matplotlib.py


## 3 Handling Data I/O Errors

No matter how unlikely it may seem, errors can still happen, especially when dealing with interfaces to the real world. They might be the result of spurious noise on a serial interface, an out-of-range voltage level on an analog input, or a fault in an external instrument. How the software detects and handles errors is directly related to its robustness. Another way to put it would be to say that robust software tends to exhibit a high degree of fault tolerance.

For a system (be it software, hardware, or a combination of the two) to be called **faulttolerant** implies that it has the ability to detect a fault condition, take action to correct or bypass the fault, and continue to function (perhaps at a reduced level of functionality)
instead of just crashing or abruptly halting. The ability to `continue to function at reduced levels of capability` in the presence of an increasing level of errors is called **graceful degradation**(优雅降级). Of course, if the errors continue to mount, at some point the system will eventually come to a halt, but the idea is that it will do so after giving ample notice and it will `not do it in a catastrophic fashion`.

The reality is that there are almost always faults, and most things will eventually break
or wear out. How much planning you should do for the mostly likely faults and the
resulting errors is largely down to how much of a problem a failure will create. It might
be insignificant (just ignore it and move on), or it could be a really big deal (something
might explode, catch fire, or otherwise fail to stop an impending disaster). If you’ve
done your up-front planning, as discussed in Chapter 8, you should be able to identify
the nastiest scenarios and give some thought to how your system might deal with them
should they arise.

### 3.1 Classes of errors

Errors can be grouped into two broad categories: **nonfatal** and **fatal**. A nonfatal error might be something like an intermittent communications channel, perhaps due to noise or other perturbations in the medium, or someone’s foot occasionally kicking a connector
under a desk. Depending on the speed of the system and the duration of the failure, it may be possible to continue operation without adverse effects until communications can be reestablished. Another example might be an instrument that occasionally does not respond in a timely fashion, for whatever reason. If the command or query can be retried successfully with no ill effect, the error could be considered nonfatal.
(Note that nonfatal does not mean nonannoying!)。

A **fatal** error is one that requires significant intervention if the system is to continue functioning. Lacking that, it will need to perform a complete shutdown. An example of a fatal error would be the loss of control for the primary DC power supply used in an experiment. Unless there is a backup supply available that can automatically take over, the system will need to shut down until the problem can be resolved. Another example might be the failure of the control system for the liquid nitrogen supply used for the sorption pumps on a vacuum chamber, perhaps due to a failure in the control interface electronics, or a failure in the command communications channel. In either
case, the system will begin to lose vacuum and potentially damage things like ion gauges or sputter emitters. At the very least, the current activity should be stopped until the problem is resolved.

### 3.2 Error retry and system termination

Sometimes it may make sense to retry an operation if an error is detected, perhaps after altering a parameter to compensate for the error. While this might sound clever (and it can be), it’s not something that should be done without some serious consideration of the context, cause, and consequences of the error. Blithely attempting to retry a failed operation can sometimes cause serious damage.

The` more error-detection and self-recovery capabilities` one attempts to build into a system, the `more complicated` the system becomes. This is fairly obvious, to be sure, but what isn’t obvious is how that complexity will manifest, and the subsequent implications it might have, not only for a particular subsystem, but for the system as a whole. As complexity increases, so too does the chance of new defects being introduced. Increased complexity can also increase the number of possible execution paths in the software, some of which may be unintended.

**Figure 11-11** shows a scheme for handling a data I/O error in a **fault-tolerant** fashion. While this approach may not be suitable for every application, it does show why robust or fault-tolerant software tends to be **an order of magnitude (or more) `more expensive`  to implement than something that just does the I/O operation and returns either `pass` or `fail`**. This is particularly true when performing testing to verify the fault-tolerant behavior. In **Figure 11-11**, there are `three possible paths` that can be taken should an
error occur. In addition to the I/O operation itself, each of these paths must be tested by simulating the I/O and the error context. This rigorous testing involves a lot of work,but if you need that level of robustness there really is no other way to achieve it.

![fault-tolerant fashion](./img/data-io-fault-resistant.jpg)

An interesting point to note about `Figure 11-11` is the amount of code it implies. The data I/O operation and its return code (pass or fail, perhaps) are simple and straightforward, and might take no more than a line or two of code to implement. With the error handling included in the design, the code for performing a data I/O operation will grow by anywhere from 10 to 100 times in size. `This is typical of fault-tolerant software`.A large portion of it is concerned with error detection and handling, and only a fraction actually deals directly with the I/O. Also note that the last decision block, **“Backup active?,”** means that if the backup is already in use (i.e., the test is True), there are no more options left except to fail.

When detecting and attempting to deal with an error, the system has to make a decision as to whether to attempt to recover from the error (and what recovery strategy to use)or just try to shut down gracefully. The logic making that decision must have inputs in the form of data describing the context in which the error occurred and the current state of the system, and there may also be a need to define excluded operations that should not be used.

For example, it may not be a good idea for a system controlling a pressure vessel to just
relinquish control of the system without first performing some kind of check to determine
if the pressure needs to be released. If the pressure continues to build even after
the pumps and heaters are disabled (this can happen), there is a risk that the vessel may
explode, especially if the error involved an over-pressure-related situation to start with.
A graceful shutdown could possibly involve some type of venting action before control
is completely terminated


Similarly, if an error occurs in a system that is moving a mass of some type, does it make sense for the system to just stop? If the action of lifting or moving the mass entails control of power to a motor or servo, it might not be a good idea to just kill the power 

The system may need to engage some type of braking or locking mechanism, or it might make sense for the mass to be lowered to a safe position prior to shutdown (if possible).

These considerations also come into play when attempting to retry a failed operation.Retries may not be appropriate after some types of failures, such as the loss of direct positional feedback, or the failure of a temperature sensor. Other failures may be known to be transient, and the operations can be retried some number of times before the situation is declared hopeless.

Consider the situation where the position of a secondary mechanism is dependent on the position of a primary mechanism, both of which are moving at a slow and relatively continuous rate for extended periods of time. The link between the two is a communications channel that is known to occasionally drop out due to system load or other factors. In a situation like this, the secondary mechanism that is following the primary one might be able to predict where it should be over short periods of time. This allows it to continue to function without an update from the primary mechanism. If after some period of time the communications with the primary mechanism cannot be reestablished,the secondary mechanism will enter an error condition. If it does reestablish the communications channel with the primary mechanism before the timeout period, it can update its position, if necessary, and reset the timeout


Failure analysis, which we discussed briefly in the section **“Handling Errors and Faults” on page 272 in Chapter 8**, comes into play when making decisions like these.If done correctly, it can provide the guidance needed to make the decision to terminate abruptly, terminate gracefully, or attempt to recover. Lacking a failure analysis, the best choice is often to just terminate gracefully, and provide sufficient information (typically in a crash log or something similar) to allow someone to go back and ascertain the cause of the problem later.

## Reference

Allen B. Downey. [Think OS:A Brief Introduction to Operating Systems](http://greenteapress.com/wp/think-os/)


[The Python Standard Library:Concurrent Execution](https://docs.python.org/3/library/concurrency.html)

* [threading — Thread-based parallelism](https://docs.python.org/3/library/threading.html)

* [queue — A synchronized queue class](https://docs.python.org/3/library/queue.html)


[Networking and Interprocess Communication](https://docs.python.org/3/library/ipc.html)

  * asyncio — Asynchronous I/O https://docs.python.org/3/library/asyncio.html  

Pebble： https://github.com/noxdafox/pebble


J. M. Hughes. Real World Instrumentation with Python, O'Reilly Media, December 2010


Doug Hellmann. The Python3 Standard Library by Example,Pearson Education, Inc. 2017

* http://doughellmann.com/blog/the-python-3-standard-library-by-example

* Python 3 Module of the Week https://pymotw.com/3/
   

Gabriele Lanaro. Python High Performance,Second Edition, Packt Publishing,2017


David Beazley, Brian K. Jone. Python Cookbook, Third Edition. O’Reilly Media,2013
 